In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import spatialdata_io
import spatialdata as sd

# while not pip installable, add path to file 
import sys
sys.path.append("../../src")
import troutpy 


/home/sergio/.local/lib/python3.10/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(
/home/sergio/.local/lib/python3.10/site-packages/numba/core/decorators.py:246: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)
/home/sergio/.local/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)
/home/sergio/.local/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)


## Read spatial data

In [2]:
xenium_path_cropped='/media/sergio/Discovair_final/mousebrain_prime_crop_points2regions_annotated.zarr'
sdata=sd.read_zarr(xenium_path_cropped)

/home/sergio/.local/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/home/sergio/.local/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/home/sergio/.local/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/home/sergio/.local/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/home/sergio/.local/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/home/sergio/.local/

### Overexpression 

In [3]:
control_codewords = ['negative_control_probe','unassigned_codeword', 'deprecated_codeword', 
                     'genomic_control_probe', 'negative_control_codeword']

exrna.tl.quantify_overexpression(sdata,layer='transcripts',codeword_column="codeword_category",
    control_codewords=control_codewords,gene_id_column="feature_name",percentile_threshold=99.99)


/home/sergio/.local/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Added 'xrna_metadata' table with 13035 unique genes to the SpatialData object.


/home/sergio/Jnotebooks/exRNA/spatial_exotranscriptomics/notebooks/spatial_data_tutorials/../../exrna/tl/quantify_xrna.py:239: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sdata['xrna_metadata'].var['control_probe']=sdata['xrna_metadata'].var['control_probe'].fillna(False)


### Intracellular vs extracellular expression

In [4]:
exrna.tl.extracellular_enrichment(sdata)

/home/sergio/.local/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


### Test for spatial_variability

In [5]:
exrna.tl.spatial_variability(sdata,  gene_id_key='feature_name', n_neighbors=10,binsize=20)

Extracting gene counts: 100%|██████████| 13031/13031 [00:00<00:00, 16558.67it/s]


### Test for local density

In [6]:
exrna.tl.spatial_colocalization(sdata, coords_keys=['x', 'y'], gene_id_key='feature_name',resolution=1000,binsize=5, threshold_colocalized=1 ,copy=False)

Extracting gene counts: 100%|██████████| 13031/13031 [00:07<00:00, 1787.40it/s]


### Save the output as zarr

In [7]:
sdata['xrna_metadata'].var['control_probe'].unique()

array([False,  True])

In [8]:
xenium_path_cropped='/media/sergio/Discovair_final/mousebrain_prime_crop_quantified.zarr'
sdata.write(xenium_path_cropped,overwrite=True)

INFO     The SpatialData object is not self-contained (i.e. it contains some elements that are Dask-backed from    
         locations outside /media/sergio/Discovair_final/mousebrain_prime_crop_quantified.zarr). Please see the    
         documentation of `is_self_contained()` to understand the implications of working with SpatialData objects 
         that are not self-contained.                                                                              
INFO     The Zarr backing store has been changed from                                                              
         /media/sergio/Discovair_final/mousebrain_prime_crop_points2regions_annotated.zarr the new file path:      
         /media/sergio/Discovair_final/mousebrain_prime_crop_quantified.zarr                                       


### Output_exploration

In [38]:
exrna_metrics=sdata['xrna_metadata'].var
exrna_metrics=exrna_metrics[exrna_metrics['count']>120]
exrna_metrics_filt=exrna_metrics.loc[:,['logfoldratio_over_noise','logfoldratio_extracellular','moran_I','proportion_of_colocalized']]